In [ ]:
import requests

#Nuestras credenciales
client_id = "931d7d4218dc436b9e30b0ad300d487e"
client_secret = "689208ad1ed9456ca350ece8da27468a"

#URL para obtencion de tokens
url = "https://accounts.spotify.com/api/token"
#Headers necesarios para la request
headers = {
    "Content-Type": "application/x-www-form-urlencoded"
}
#Datos a enviar en la request, aqui van las credenciales
data = {
    "grant_type": "client_credentials",
    "client_id": client_id,
    "client_secret": client_secret
}

#Hacemos la request
response = requests.post(url, headers=headers, data=data)

#Guardamos el token
access_token = response.json().get("access_token")

#Mostramos el resultado
print(access_token)

None


In [ ]:
#Biblioteca para formatear json, sólo necesaria por claridad
import json
import csv

# Lista de 20 IDs de artistas (con comentarios para identificar a cada uno)
artist_ids = [
    "1vCWHaC5f2uS3yhpwWbIA6",  # Avicii
    "06HL4z0CvFAxyc27GXpf02",  # Taylor Swift
    "3TVXtAsR1Inumwj472S9r4",  # Drake
    "66CXWjxzNUsdJxJ2JdwvnR",  # Ariana Grande
    "1uNFoZAHBGtllmzznpCI3s",  # Justin Bieber
    "0du5cEVh5yTK9QJze8zA0C",  # Bruno Mars
    "3Nrfpe0tUJi4K4DXYWgMUX",  # BTS
    "7dGJo4pcD2V6oG8kP0tJRR",  # Eminem
    "1Xyo4u8uXC1ZmMpatF05PJ",  # The Weeknd
    "4gzpq5DPGxSnKTe4SA8HAU",  # Coldplay
    "246dkjvS1zLTtiykXe5h60",  # Post Malone
    "7CajNmpbOovFoOoasH2HaY",  # Calvin Harris
    "1u7kkVrr14iBvrpYnZILJR",  # David Guetta
    "6eUKZXaKkcviH0Ku9w2n3V",  # Ed Sheeran
    "4AK6F7OLvEQ5QYCBNiQWHq",  # Elton John
    "5pKCCKE2ajJHZ9KAiaK11H",  # Rihanna
    "2wY79sveU1sp5g7SokKOiI",  # Sam Smith
    "4nDoRrQiYLoBzwC5BhVJzF",  # Camila Cabello
    "4VMYDCV2IEDYJArk749S6m",  # Daddy Yankee
    "5K4W6rqBFWDnAN6FQUkS6x"   # Kanye West
]

artists_attributes_list = []

# Iteramos sobre cada id para hacer la consulta y extraer la información
for artist_id in artist_ids:
    url_artist = f"https://api.spotify.com/v1/artists/{artist_id}"
    response_artist = requests.get(url_artist, headers=headers_artist)
    
    if response_artist.status_code != 200:
        print(f"Error al obtener datos para el artista con ID: {artist_id}")
        continue  # Si hay error, pasa al siguiente artista
    
    artist_data = response_artist.json()
    
    # Obtenemos listas de imágenes y géneros (si existen)
    images = artist_data.get("images", [])
    genres = artist_data.get("genres", [])
    
    # Extraemos 15 atributos (algunos directos, otros derivados)
    artist_attributes = {
        "id": artist_data.get("id"),                                            # 1. Identificador del artista
        "name": artist_data.get("name"),                                        # 2. Nombre del artista
        "type": artist_data.get("type"),                                        # 3. Tipo (ej.: artist)
        "uri": artist_data.get("uri"),                                          # 4. URI de Spotify
        "href": artist_data.get("href"),                                        # 5. Enlace a la API
        "popularity": artist_data.get("popularity"),                            # 6. Popularidad
        "genres": genres,                                                       # 7. Lista de géneros
        "primary_genre": genres[0] if genres else None,                         # 8. Primer género (si existe)
        "external_url": artist_data.get("external_urls", {}).get("spotify"),      # 9. URL externa en Spotify
        "followers_total": artist_data.get("followers", {}).get("total"),         # 10. Total de seguidores
        "followers_href": artist_data.get("followers", {}).get("href"),           # 11. Href de seguidores (usualmente None)
        "number_of_images": len(images),                                        # 12. Cantidad de imágenes disponibles
        "first_image_url": images[0].get("url") if images else None,              # 13. URL de la primera imagen
        "last_image_url": images[-1].get("url") if images else None,              # 14. URL de la última imagen
        "images_info": [                                                        # 15. Detalles de cada imagen (url, height y width)
            {"url": img.get("url"), "height": img.get("height"), "width": img.get("width")}
            for img in images
        ]
    }
    
    artists_attributes_list.append(artist_attributes)

# Mostramos la lista de diccionarios en formato JSON


# Nombre del archivo CSV donde se almacenarán los datos
csv_filename = "artists_data.csv"

# Definimos los nombres de las columnas que estarán en el archivo CSV
fieldnames = [
    "id", "name", "type", "uri", "href", "popularity",
    "genres", "primary_genre", "external_url", "followers_total",
    "followers_href", "number_of_images", "first_image_url", "last_image_url",
    "images_info"
]

# Abrimos el archivo CSV en modo escritura
with open(csv_filename, mode="w", newline="", encoding="utf-8") as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    
    # Escribimos la fila de encabezado
    writer.writeheader()
    
    # Escribimos cada fila con los datos de los artistas
    for artist in artists_attributes_list:
        # Convertimos la lista de géneros en una cadena separada por comas
        artist["genres"] = ", ".join(artist["genres"])
        
        # Convertimos la lista de imágenes en una cadena con detalles separados por ";"
        artist["images_info"] = "; ".join([
            f"url: {img['url']}, height: {img['height']}, width: {img['width']}" 
            for img in artist["images_info"]
        ])
        
        # Escribimos la fila de datos del artista
        writer.writerow(artist)

print(f"Datos almacenados en {csv_filename}")

